# Music genre prediction using Theano


In this lesson we will use the the same approach as in previous one but scale it up to solve a real-life problem.

## Challenge

This time we want to solve a challenging classification task. Given a 3-second single-channel audio excerpt sampled at 400Hz we want to tell if is a piece of classical or rock music.

For the purpose of this exercise I have prepared a dataset that consists of 10k audio excerpts. This is a subset of [Google AudioSet](https://research.google.com/audioset/). The process of gathering the data was simple:
* download YouTube videos that were tagged with particular music genre from the Internet and extract audio stream
* cut the segment of the audio indicated in the segments file (usually 10 seconds)
* cut middle 3 seconds second of that segment and resmaple at 400Hz
This produces audio excerpts that have exactly 1200 samples. Please note that 400Hz sampling rate is quite low and therefore the audio contains only the lowest frequencies (deep, low and mid bass). Additionally the quality of some of the videos is quite poor and some videos are mislabeled which all makes the challenge even more difficult.

The 10k excerpts consists of exactly 5k classical music pieces and 5k rock music pieces. I have split them into training (80%) and testing (20%) subsets. Therefore training set consists of exactly 4k classical and 4k rock pieces and test set contains 1k examples from each class. The dataset is available as a single \*.npz file. The file weights around 48MB, it should be placed in the same directory as the script. The link is provided in the top-level README of this tutorial.

Here are some random examples of classical music excerpts:
![alt text](samples_classical.png "classical music")
and rock music excerpts:
![alt text](samples_rock.png "rock music")

## Approach

To solve the challenge we will use the same approach as in previous lesson, but the function we will be using (aka model) needs to be much bigger. A common choice is to use neural network, or in other words [multilayer perceptron](https://en.wikipedia.org/wiki/Multilayer_perceptron).

## Implementation

In [1]:
import random
import theano
import numpy as np

IN_FILE = "./classical_vs_rock.npz"
SAMPLE_WIDTH = 1200

BATCH_SIZE = 16
LEARNING_RATE = 0.01

NUM_UPDATES = 5000
EVAL_STEP = 500

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


There are some new meta-parameters of our algorithm, let's describe them here:

Batch size is the number of examples we will be using during single training step (single weight update step). We like to process examples in batches because that stabilizes the learning process, in short it helps the weights to move into the right direction rather than random. Think of this in following way: if weight update improves results for 16 examples at the same time it is far more probable that it will improve results for other examples as well. If we did an update based only on single example, the probability that the update will be random (will not generalize to other examples) is far greater.

Please note that we have reduced the learning rate significantly. Because we will have so many parameters, we want them to be updated slowly and steadily to smooth the possible spikes in gradient that might destabilize the training process.

Because learning rate is much smaller, we have to do much more updates. With this particular number of updates (5k) each training example will be used 10 times on average (8000/16 == 500).

To keep track of progress of the training we will do evaluation on test set every 500 updates (ten times during the whole training process).

First we will wrap data handling into a separate class:

In [2]:
class DataServer(object):
    def __init__(self):
        data = np.load(IN_FILE)
        self.train = data['train'].astype(np.float32)
        self.test = data['test'].astype(np.float32)
        assert(self.train.shape[1] == SAMPLE_WIDTH)

        self.test_targets = np.zeros((self.test.shape[0]), dtype=np.float32)
        self.test_targets[self.test_targets.shape[0]/2:] = 1.0

    def get_test(self):
        return self.test, self.test_targets

    def get_train_sample(self, batch_size):
        inputs = np.empty((batch_size, self.train.shape[1]))
        targets = np.zeros((batch_size))

        for idx in range(batch_size):
            train_idx = random.randrange(self.train.shape[0])
            inputs[idx] = self.train[train_idx]
            if train_idx >= self.train.shape[0]/2:
                targets[idx] = 1.0

        return inputs.astype(np.float32), targets.astype(np.float32)

This class provides two methods: one to be used during training phase and one for testing phase. Both methods return two NumPy arrays: one with inputs and one with expected outputs. Expected outputs are not given directly in the \*.npz file but the structure of the file is simple therefore we can recreate them: first half of the examples is always for class 0 (classical music), second half contains rock music excerpts (class 1).

The get_test function always returns the whole test set at once, the get_train_sample function selects random batch_size examples. In the returned NumPy arrays with inputs, first dimension is the number of examples (2k for test and 16 for train) and the second is number of samples per example, e.i. 1200.

We will also define our parameter initializer as a separate function to keep the code clean:

In [3]:
def init_values(shape):
    values = np.random.normal(scale=0.1, size=shape)
    return values.astype(np.float32)

In previous example we initialized our internal parameters (aka weights) with fixed numbers (0.2 and 0.7). When the number of parameters grows, a good idea is to initialize them with small random numbers. This will be a good seed to start training from.

Now it's finally time to define our model. We already said we want to use the multilayer perceptron architecture. To implement it we first need to take a closer look how [single neuron looks like](https://en.wikipedia.org/wiki/Artificial_neuron).

OK, so a single neuron has a weight for each neuron in previous layer (or input value if it is the first layer after input). This can be expressed as a element-wise multiplication of two vectors followed by a sum and a transfer function.
For N neurons this can be expressed as a matrix multiplication of inputs and weights according to following scheme:
```
(batch_size, num_inputs) * (num_inputs, num_neurons) = (batch_size, num_neurons)
```
For the first layer after input the num_inputs if of course the SAMPLE_WIDTH.

Additional to the weighted sum of inputs, we also often add a constant value to each neuron. This helps to set appropriate level of activation of a neuron for the following activation function, which in our case will be a standard [rectifier](https://en.wikipedia.org/wiki/Rectifier_(neural_networks).

All in all, our first layer will look like this:

In [4]:
x = theano.tensor.fmatrix('x')
target = theano.tensor.fvector('target')

W1 = theano.shared(init_values((SAMPLE_WIDTH, 256)), 'W1')
B1 = theano.shared(init_values((1, 256)), 'B1', broadcastable=(True, False))
y = theano.tensor.dot(x, W1) + B1
y = theano.tensor.switch(theano.tensor.gt(y, 0.0), y, 0.0)

This way we have defined a layer of 256 neurons with biases, all initialized with small random values. There are two sets of parameters: W1 and B1. To make sure that the biases will be added for all examples we needed to tell Theano that first dimension of B1 is broadcastable - the same values should be repeated for all examples (repeated batch_size times).

One layer is not enough to encompass any higher-level logic, so we add two more layers with 64 and 1 neuron respectively. The final neuron should return 0 for classical music and 1 for rock music piece.

In [5]:
W2 = theano.shared(init_values((256, 64)), 'W2')
B2 = theano.shared(init_values((1, 64)), 'B2', broadcastable=(True, False))
y = theano.tensor.dot(y, W2) + B2
y = theano.tensor.switch(theano.tensor.gt(y, 0.0), y, 0.0)

W3 = theano.shared(init_values((64, 1)), 'W3')
B3 = theano.shared(init_values((1, 1)), 'B3', broadcastable=(True, False))
y = theano.tensor.dot(y, W3) + B3
y = theano.tensor.flatten(y)
y = theano.tensor.nnet.nnet.sigmoid(y)

Please note that the final neuron has sigmoid transfer function instead of rectifier. [Sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function) helps in classification problems because the network doesn't need to produce exactly 0.0 or 1.0, but can produce big negative values for class "0" and big positive values for class "1". As long as the values are big enough - loss will be approaching zero (assuming the network makes the right decision of course).

You might have also noticed a 'flatten' function. It's purpose is very simple. After final layer our output shape will be (batch_size, 1). Because our expected outputs have only one axis, we remove the now redundant axis from predictions.

From now on the code should look familiar:

In [6]:
#Cost function
cost = theano.tensor.sqr(target - y)
cost = theano.tensor.mean(cost)

#Gradients
params = [W1, B1, W2, B2, W3, B3]
gradients = theano.tensor.grad(cost, params)

#Updates
updates = list()
for param_idx, param in enumerate(params):
    updated_param = param - gradients[param_idx] * LEARNING_RATE
    updates.append((param, updated_param))

#Train function doesn't reutn anything -  just performs updates
train = theano.function([x, target], updates=updates)
#Test function only returns predictions for given input
test = theano.function([x], y)

This time we have six sets of parameters, therefore our "gradients" as well as "updates" are both lists of length 6.

We also compile two functions, separate for training and separate for testing. The training function needs both inputs and expected outputs to perform updates, but it doesn't return anything (of course it could if we wanted). Testing functions doesn't perform updates and therefore it doesn't need expected outputs. Instead it returns predicted values. We will be calculating network's performance using NumPy, we could also implement this in Theano and get the accuracy as the function output, but let's keep it as simple as possible.

When calculating performance we simply compare predictions with expected output and count all elements where there was a match and divide this number by total number of examples.

Now all we have to do is run training loop:

In [7]:
ds = DataServer()
for i in xrange(NUM_UPDATES):
    inputs, targets = ds.get_train_sample(BATCH_SIZE)
    train(inputs, targets)

    #Evaluation
    if (i + 1) % EVAL_STEP == 0:
        inputs, targets = ds.get_test()
        predictions = test(inputs)

        predictions = predictions.round().clip(0, 1)
        performance = ((predictions == targets)*1.0).sum()/float(targets.shape[0])
        print "Accuracy after update {}/{}: {}%".format(i + 1, NUM_UPDATES, float(performance)*100)

Accuracy after update 500/5000: 81.1%
Accuracy after update 1000/5000: 83.3%
Accuracy after update 1500/5000: 83.05%
Accuracy after update 2000/5000: 83.15%
Accuracy after update 2500/5000: 83.05%
Accuracy after update 3000/5000: 83.2%
Accuracy after update 3500/5000: 83.2%
Accuracy after update 4000/5000: 83.3%
Accuracy after update 4500/5000: 83.55%
Accuracy after update 5000/5000: 83.35%


Nice! As we can see the network started training really fast, just after first batch of updates the accuracy was already above 80%. Subsequent updates improved it but only slightly.

The final thing we want to do it to get the weights of our trained model so that we can use it somewhere else. To do this I will compile special function that doesn't require any inputs, just returns the weights:

In [8]:
params = theano.function([], params)
params = params()
for param_idx, param in enumerate(params):
    print "Parameter {}: type: {} shape: {}".format(param_idx, param.dtype, param.shape)

Parameter 0: type: float32 shape: (1200, 256)
Parameter 1: type: float32 shape: (1, 256)
Parameter 2: type: float32 shape: (256, 64)
Parameter 3: type: float32 shape: (1, 64)
Parameter 4: type: float32 shape: (64, 1)
Parameter 5: type: float32 shape: (1, 1)


Voila, we how a working model!

Its performance reaches 83% which seems quite OK. But can we do better?

We surely can, but to do that we will need a more sophisticated network architecture. But that would require a lot of coding in Theano, therefore we will use a tool that has all required building blocks already implemented.